In [1]:
from datetime import datetime, timezone
import math
import os

import xarray as xr
import yaml
from cf_units import Unit

import data_catalog
from tseries_utils import clean_units, get_weight, tseries_fname, tseries_copy_vars

In [2]:
tseries_specs_fname = 'tseries_specs_atm.yaml'
with open(tseries_specs_fname, mode='r') as fptr:
    tseries_specs = yaml.load(fptr)

catalog_name = 'experiments'
data_catalog.set_catalog(catalog_name)

varname = 'SFCO2_OCN'
ts_spec = tseries_specs[varname]
print(ts_spec)

component = 'atm'
stream = 'cam.h0'
experiment = '1pctCO2-bgc'
entries = data_catalog.find_in_index(
    variable=varname, component=component, stream=stream, experiment=experiment)
entries

active catalog: experiments
{'component': 'atm', 'stream': 'cam.h0', 'weight_op': 'integrate', 'reduce_dims': ['lat', 'lon'], 'unit_conv': '(12 g)/(44 g)', 'units_out': 'Pg yr-1'}


,case,component,ctrl_branch_year,ctrl_experiment,date_range,ensemble,experiment,file_basename,files,sequence_order,stream,variable,grid,year_offset,has_ocean_bgc
51094,b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001,atm,501.0,piControl,"['000101', '005101']",0,1pctCO2-bgc,b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001.cam....,/glade/scratch/klindsay/archive/b.e21.B1PCT.f0...,0,cam.h0,SFCO2_OCN,NaN,NaN,NaN
51095,b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001,atm,501.0,piControl,"['005102', '008012']",0,1pctCO2-bgc,b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001.cam....,/glade/scratch/klindsay/archive/b.e21.B1PCT.f0...,0,cam.h0,SFCO2_OCN,NaN,NaN,NaN


In [3]:
ensemble = 0
fnames = data_catalog.get_files(
    variable=varname, component=component, stream=stream, experiment=experiment, ensemble=ensemble)
fnames

['/glade/scratch/klindsay/archive/b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001/atm/proc/tseries/month_1/b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001.cam.h0.SFCO2_OCN.000101-005101.nc',
 '/glade/scratch/klindsay/archive/b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001/atm/proc/tseries/month_1/b.e21.B1PCT.f09_g17.CMIP6-1pctCO2-bgc.001.cam.h0.SFCO2_OCN.005102-008012.nc']

In [4]:
ds_in = xr.open_mfdataset(fnames, decode_times=False, decode_coords=False, data_vars='minimal')
ds_in

<xarray.Dataset>
Dimensions:       (ilev: 33, lat: 192, lev: 32, lon: 288, nbnd: 2, time: 960, zlon: 1)
Coordinates:
  * lat           (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * zlon          (zlon) float64 0.0
  * lon           (lon) float64 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * time          (time) float64 31.0 59.0 90.0 ... 2.914e+04 2.917e+04 2.92e+04
Dimensions without coordinates: nbnd
Data variables:
    zlon_bnds     (zlon, nbnd) float64 0.0 358.8
    gw            (lat) float64 3.382e-05 0.0002705 ... 0.0002705 3.382e-05
    hyam          (lev) float64 0.003643 0.007595 0.01436 ... 0.001989 0.0
    hybm          (lev) float64 0.0 0.0 0.0 0.0 ... 0.9251 0.9512 0.9743 0.9926
    P0            float64 1e+05
    hyai          (ilev) float64 0.002255 0.005032 0.01016 ... 0.003979 0.0 0.0
    h

In [5]:
da_in = ds_in[varname]
da_in

<xarray.DataArray 'SFCO2_OCN' (time: 960, lat: 192, lon: 288)>
dask.array<shape=(960, 192, 288), dtype=float32, chunksize=(600, 192, 288)>
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) float64 31.0 59.0 90.0 ... 2.914e+04 2.917e+04 2.92e+04
Attributes:
    units:         kg/m2/s
    long_name:     CO2_OCN surface flux
    cell_methods:  time: mean

In [6]:
var_units = clean_units(da_in.attrs['units'])
print(var_units)
if 'unit_conv' in ts_spec:
    var_units = '(%s)(%s)' % (str(ts_spec['unit_conv']), var_units)
print(var_units)

kg/m2/s
((12 g)/(44 g))(kg/m2/s)


In [7]:
reduce_dims = ts_spec['reduce_dims']
weight = get_weight(ds_in, component, reduce_dims)
print(weight)
area_earth_wikipedia = 510072000*1.0e3**2
print(weight.sum(dim=('lat', 'lon')).values)
print(weight.sum(dim=('lat', 'lon')).values / area_earth_wikipedia)

<xarray.DataArray (lat: 192, lon: 288)>
array([[2.994837e+07, 2.994837e+07, 2.994837e+07, ..., 2.994837e+07,
        2.994837e+07, 2.994837e+07],
       [2.395748e+08, 2.395748e+08, 2.395748e+08, ..., 2.395748e+08,
        2.395748e+08, 2.395748e+08],
       [4.790848e+08, 4.790848e+08, 4.790848e+08, ..., 4.790848e+08,
        4.790848e+08, 4.790848e+08],
       ...,
       [4.790848e+08, 4.790848e+08, 4.790848e+08, ..., 4.790848e+08,
        4.790848e+08, 4.790848e+08],
       [2.395748e+08, 2.395748e+08, 2.395748e+08, ..., 2.395748e+08,
        2.395748e+08, 2.395748e+08],
       [2.994837e+07, 2.994837e+07, 2.994837e+07, ..., 2.994837e+07,
        2.994837e+07, 2.994837e+07]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    units:    m2
510099699070761.6
1.0000543042369736


In [8]:
da_out = (da_in * weight).sum(dim=reduce_dims)
da_out.name = varname
da_out.attrs['long_name'] = 'Integrated '+da_in.attrs['long_name']
da_out.attrs['units']=Unit('(%s)(%s)' % (weight.attrs['units'], var_units)).format()
da_out

<xarray.DataArray 'SFCO2_OCN' (time: 960)>
dask.array<shape=(960,), dtype=float64, chunksize=(600,)>
Coordinates:
  * time     (time) float64 31.0 59.0 90.0 ... 2.914e+04 2.917e+04 2.92e+04
Attributes:
    long_name:  Integrated CO2_OCN surface flux
    units:      0.272727272727273 kg.s-1

In [9]:
Unit(da_out.attrs['units']).convert(da_out.values[0:12], Unit(clean_units(ts_spec['units_out'])))

array([ 0.27223619,  0.66236102,  0.72936225,  0.40835357, -0.0800955 ,
       -0.05985384,  0.30330485,  0.23189009, -0.06107542, -0.41818962,
       -0.83410069, -0.83703803])

In [10]:
ds_out = da_out.to_dataset()
merge_objs = []
if 'bounds' in ds_in['time'].attrs:
    tb = ds_in[ds_in['time'].attrs['bounds']]
    tb.attrs['units'] = ds_in['time'].attrs['units']
    tb.attrs['calendar'] = ds_in['time'].attrs['calendar']
    ds_out = xr.merge((ds_out, tb))
print(ds_out)
for copy_var in tseries_copy_vars(component):
    print(copy_var)
    ds_out = xr.merge((ds_out, ds_in[copy_var]))
ds_out.attrs = ds_in.attrs
datestamp = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S %Z")
ds_out.attrs['history'] = 'created at %s' % datestamp
print(ds_out)

<xarray.Dataset>
Dimensions:    (nbnd: 2, time: 960)
Coordinates:
  * time       (time) float64 31.0 59.0 90.0 ... 2.914e+04 2.917e+04 2.92e+04
Dimensions without coordinates: nbnd
Data variables:
    SFCO2_OCN  (time) float64 dask.array<shape=(960,), chunksize=(600,)>
    time_bnds  (time, nbnd) float64 dask.array<shape=(960, 2), chunksize=(600, 2)>
co2vmr
ch4vmr
f11vmr
f12vmr
n2ovmr
sol_tsi
<xarray.Dataset>
Dimensions:    (nbnd: 2, time: 960)
Coordinates:
  * time       (time) float64 31.0 59.0 90.0 ... 2.914e+04 2.917e+04 2.92e+04
Dimensions without coordinates: nbnd
Data variables:
    SFCO2_OCN  (time) float64 dask.array<shape=(960,), chunksize=(600,)>
    time_bnds  (time, nbnd) float64 dask.array<shape=(960, 2), chunksize=(600, 2)>
    co2vmr     (time) float64 dask.array<shape=(960,), chunksize=(600,)>
    ch4vmr     (time) float64 dask.array<shape=(960,), chunksize=(600,)>
    f11vmr     (time) float64 dask.array<shape=(960,), chunksize=(600,)>
    f12vmr     (time) float64 da